In [1]:
!pip install boto3

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.6/135.6 kB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.6/79.6 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.7/10.7 MB 41.2 MB/s eta 0:00:00


In [2]:
PROFILING_GROUP_NAME = 'DemoApplication-WithIssues'
CSV_FILE_NAME = 'recommendations.csv'

In [3]:
import boto3
from botocore.config import Config
from datetime import datetime
my_config = Config(
    region_name = 'eu-north-1',
    retries = {
        'max_attempts': 10,
        'mode': 'standard'
    }
)

client = boto3.client('codeguruprofiler', 
                      config=my_config, 
                      aws_access_key_id='<aws_access_key_id>',
                      aws_secret_access_key='<aws_secret_access_key>'
)

In [4]:
fields = {
    'id': 'Id',
    'name': 'Name',
    'description': 'Description',
    'resolutionSteps': 'Resolution',
    'startTime': 'Profile start time',
    'endTime': 'Profile end time'
}

In [5]:
import csv

rcmValues = []

response = client.list_findings_reports(
    dailyReportsOnly=False,
    endTime=datetime.now(),
    profilingGroupName=PROFILING_GROUP_NAME,
    startTime=datetime(2023, 1, 1)
)

for summary in dict(response)['findingsReportSummaries']:
  print(summary)
  print('\t' + str(summary['totalNumberOfFindings']))
  if summary['totalNumberOfFindings'] > 0:
    rcms = client.get_recommendations(
      endTime=summary['profileEndTime'],
      locale='en-US',
      profilingGroupName=PROFILING_GROUP_NAME,
      startTime=summary['profileStartTime']
    )

    for rcm in dict(rcms)['recommendations']:
      pattern = rcm['pattern']
      del rcm['pattern']
      rcm = rcm | pattern
      rcmVal = []
      for field in fields:
        print(field)
        rcmVal.append(rcm[field])
      rcmValues.append(rcmVal)

with open(CSV_FILE_NAME, 'w') as f:
    write = csv.writer(f)
      
    write.writerow(fields.values())
    write.writerows(rcmValues)

{'id': 'f8053456-b79e-41f2-8538-bdcc2335c143', 'profileEndTime': datetime.datetime(2023, 4, 27, 0, 0, tzinfo=tzlocal()), 'profileStartTime': datetime.datetime(2023, 4, 26, 0, 0, tzinfo=tzlocal()), 'profilingGroupName': 'DemoApplication-WithIssues', 'totalNumberOfFindings': 0}
	0
{'id': '4247d780-a5d5-46bf-a097-e24aad464b29', 'profileEndTime': datetime.datetime(2023, 4, 26, 13, 0, tzinfo=tzlocal()), 'profileStartTime': datetime.datetime(2023, 4, 26, 12, 0, tzinfo=tzlocal()), 'profilingGroupName': 'DemoApplication-WithIssues', 'totalNumberOfFindings': 0}
	0
{'id': 'a3dab4a8-658f-47ac-a69b-c1ea1ac01d1a', 'profileEndTime': datetime.datetime(2023, 4, 23, 0, 0, tzinfo=tzlocal()), 'profileStartTime': datetime.datetime(2023, 4, 22, 0, 0, tzinfo=tzlocal()), 'profilingGroupName': 'DemoApplication-WithIssues', 'totalNumberOfFindings': 1}
	1
id
name
description
resolutionSteps
startTime
endTime
{'id': '5de5d713-6502-400b-b6cf-43dde139915a', 'profileEndTime': datetime.datetime(2023, 4, 22, 18, 0, t

In [ ]:
rcmTable = [list(fields.values())] + rcmValues
print(', '.join(rcmValues[0]))

TypeError: ignored

In [ ]:
import requests

ENDPOINT = 'https://eu2.ca.analytics.ibm.com'

payload = {
  "parameters": [
    {
      "name": "CAMAPILoginKey",
      "value": "AWkxREEwNEY2MDYzREU0NTQxOUQ4QkQyMkU3MDE3NzI2QhPX601ozyLBUdJnff6twg1oDscu"
    }
  ]
}

headers = {
    'IBM-BA-Authorization': "CAM MTsxMDE6YWQ2MmJlZWYtZGI2NS00NjdjLWJkMzctZjhlMDkxMDY5NjM2OjM0NDM2MjE1NTg7MDszOzA7",
}

r = requests.put(ENDPOINT + '/api/v1/session', json=payload)


print(dict(r.headers).keys())

dict_keys(['Date', 'Content-Type', 'Content-Length', 'Connection', 'Cache-Control', 'Content-Language', 'Expires', 'Set-Cookie', 'Strict-Transport-Security', 'X-CA-Affinity', 'X-CA-Transaction', 'X-Content-Type-Options', 'Content-Security-Policy'])


In [ ]:
apiKey = dict(r.cookies)['cam_passport']
print(apiKey)

MTsxMDE6MzRkNTc2OGMtYjNlYi1hMTg5LTg1NmItZDdlOWI3NTcyZjllOjIzODg2ODYyNzk7MDszOzA7


In [ ]:
headers = {
    'IBM-BA-Authorization': "CAM " + apiKey,
}

In [ ]:
stamp = datetime.now().strftime('%Y%m%d-%H%M%S')

payload = {
  "destination": "i30F03D682021444F8D2368C127AC0ABD",
  "filename": "recommendation_report-" + stamp
}

r = requests.post(ENDPOINT + '/api/v1/files/import', json=payload, headers=headers)
print(r.status_code)
importPath = eval(r.text)['importPath']
print(importPath)
affinity = dict(r.headers)['X-CA-Affinity']
print(affinity)

202
/api/v1/files/import/segment/1683033748137
563143633


In [ ]:
upload_headers = headers.copy()
upload_headers['X-CA-Affinity'] = affinity
upload_headers['content-type'] = 'text/csv'
upload_headers['accept'] = 'application/json'
#upload_headers['Content-type'] = 'application/octet-stream'
print(upload_headers)

{'IBM-BA-Authorization': 'CAM MTsxMDE6MzRkNTc2OGMtYjNlYi1hMTg5LTg1NmItZDdlOWI3NTcyZjllOjIzODg2ODYyNzk7MDszOzA7', 'X-CA-Affinity': '563143633', 'content-type': 'text/csv', 'accept': 'application/json'}


In [ ]:
#with open('Test.csv', 'rb') as csv_file:
#print(upload_headers)
r = requests.put(ENDPOINT + importPath + '?index=1', data=open(CSV_FILE_NAME, 'rb'), headers=upload_headers)
r = requests.put(ENDPOINT + importPath + '?index=-1', data='', headers=upload_headers)
print(r.text)
print(r.status_code)
print(r.headers)
taskID = eval(r.text)['taskID']

{"href":"/tasks/task2531682595829385","taskID":"task2531682595829385"}
202
{'Date': 'Thu, 27 Apr 2023 11:43:49 GMT', 'Content-Type': 'application/json', 'Content-Length': '64', 'Connection': 'keep-alive', 'Cache-Control': 'no-cache, no-store', 'Content-Encoding': 'gzip', 'Content-Language': 'en-US', 'Expires': 'Thu, 01 Jan 1970 00:00:00 GMT', 'Strict-Transport-Security': 'max-age=15724800; includeSubDomains', 'Via': '1.1 localhost (Apache-HttpClient/4.5.13 (cache))', 'X-CA-Affinity': '563143633', 'X-CA-Transaction': 'jj9hjvlMdG4lqMhlMyj22hlyC9C884jyhvsd4yq9', 'x-ca-requesttime': 'path=files/import/segment/1683033748137; start=1682595829095; end=1682595829391; elapsed=296'}


In [ ]:
new_headers = headers.copy()
new_headers['X-CA-Affinity'] = affinity
r = requests.get(ENDPOINT + '/api/v1/files/import/tasks/' + taskID, headers=new_headers)
print(r.status_code)
print(r.text)

200
{"state":"SUCCESS","response":{"href":"/base_modules/iD2E8112CADF2423A8A6DDAD5C2364471","id":"iD2E8112CADF2423A8A6DDAD5C2364471","ids":["iD2E8112CADF2423A8A6DDAD5C2364471"],"label":"recommendation_report-20230427-114343","lastModified":"2023-04-27T11:43:49.652Z"},"taskType":"FileUpload","taskID":"task2531682595829385","cmUpdateTime":0}
